<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/case-study-5--predicting-future-friendships/01_graph_theory_and_network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##An introduction to graph theory and network analysis

Our goal is to extract common data science skills from the downloaded job postings.
Then we’ll compare these skills to our resume to determine which skills are
missing. 

We will do so as follows:

1. Parse all text from the downloaded HTML files.
2. Explore the parsed output to learn how job skills are described in online
postings. We’ll pay particular attention to whether certain HTML tags are
more associated with skill descriptions.
3. Attempt to filter any irrelevant job postings from our dataset.
4. Cluster job skills based on text similarity.
5. Visualize the clusters using word clouds.
6. Adjust clustering parameters, if necessary, to improve the visualized output.
7. Compare the clustered skills to our resume to uncover missing skills.

##Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import re
import glob
import time
import numpy as np
import pandas as pd

from collections import defaultdict
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

from wordcloud import WordCloud

from bs4 import BeautifulSoup as bs

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

In [ ]:
%%shell

wget https://github.com/rahiakela/data-science-research-and-practice/raw/main/data-science-bookcamp/case-study-4--job-resume-improvement/job_postings.zip
wget https://github.com/rahiakela/data-science-research-and-practice/raw/main/data-science-bookcamp/case-study-4--job-resume-improvement/resume.txt
wget https://github.com/rahiakela/data-science-research-and-practice/raw/main/data-science-bookcamp/case-study-4--job-resume-improvement/table_of_contents.txt

unzip -qq job_postings.zip
rm -rf job_postings.zip

##Extracting skills from job posting